In [2]:
!pip install --upgrade pip
!pip install transformers==4.40.1 datasets==2.19.0 sentence-transformers==2.7.0 faiss-cpu==1.8.0 llama-index==0.10.34 llama-index-embeddings-huggingface==0.2.0 -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


# **문장 임베딩을 활용한 단어 간 유사도 계산**

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

smodel = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
dense_embeddings = smodel.encode(['학교', '공부', '운동'])
cosine_similarity(dense_embeddings) #코사인 유사도
# array([[1.0000001 , 0.5950744 , 0.32537547],
#       [0.5950744 , 1.0000002 , 0.54595673],
#       [0.32537547, 0.54595673, 0.99999976]], dtype=float32)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


array([[0.99999994, 0.59507436, 0.32537556],
       [0.59507436, 1.        , 0.54595697],
       [0.32537556, 0.54595697, 0.99999994]], dtype=float32)

# **원핫 인코딩의 한계**

In [5]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

word_dict = {"school": np.array([[1, 0, 0]]), "study": np.array([[0, 1, 0]]), "workout": np.array([[0, 0, 1]])}

# 두 단어 사이의 코사인 유사도 계산하기
# cosine_similarity(A, B) -> 코사인 유사도 측정 메서드
cosine_school_study = cosine_similarity(word_dict['school'], word_dict['study']) # 0
cosine_school_workout = cosine_similarity(word_dict['school'], word_dict['workout']) # 0